In [2]:
### Load library -----
library(DESeq2)
library(tidyverse)


In [3]:
### import GO DB -----
#Import GO database
GOdata<-read.csv('Reference csv/Cleaned_GO_4.csv', sep=';', header=FALSE)
GOdata$Gene<-as.character(GOdata$V1)
GOdata$V1<-NULL
GOdata$geneabb<-as.character(GOdata$V2)
GOdata$V2<-NULL
GOdata$genename<-as.character(GOdata$V3)
GOdata$V3<-NULL
GOdata$GO<-as.character(GOdata$V4)
GOdata$V4<-NULL

### featureCount data import -----
#Importing output of featurecounts
countdata_flg22 <- read.table("FeatureCounts/03.DMSO_flg22_WT_Leaf", header=TRUE, row.names='Geneid' )
#remove first 5 lines which is useless
countdata_flg22 <- countdata_flg22[ ,6:ncol(countdata_flg22)]
#remove .bam in the sample name
colnames(countdata_flg22) <- gsub("\\_a.bam$", "", colnames(countdata_flg22))
rownames(countdata_flg22) <- gsub("gene:", "", rownames(countdata_flg22))
colnames(countdata_flg22)
#Import sample information
saminfo_flg22 <- read.csv("Saminfo/03.Saminfo_DMSO_flg22_WT_Leaf.csv")
#change column names using saminfo
colnames(countdata_flg22) <- saminfo_flg22$condition
colnames(countdata_flg22)
#sort by column names
saminfo_flg22 <- arrange(saminfo_flg22, index)
countdata_flg22 <- countdata_flg22[,c(saminfo_flg22$index)]

#Select DMSO and flg22 only
countdata_flg22_df <- as.data.frame(countdata_flg22)
countdata_flg22_df <- countdata_flg22_df %>%
  dplyr::select(starts_with('DMSO'), starts_with('Z00_flg22'))
colnames(countdata_flg22_df)
#change it to matrix
countdata_flg22 <- as.matrix(countdata_flg22_df)
head(countdata_flg22)

[1] "X133941.bam" "X133942.bam" "X133943.bam" "X133944.bam" "X133945.bam"
 [6] "X133946.bam" "X133947.bam" "X133949.bam" "X133950.bam" "X133951.bam"
[11] "X133952.bam" "X133954.bam" "X133955.bam" "X133956.bam" "X133957.bam"
[16] "X133961.bam" "X133964.bam" "X133965.bam" "X133966.bam" "X133967.bam"
[21] "X133968.bam" "X133969.bam" "X133970.bam" "X133971.bam" "X133975.bam"
[26] "X133976.bam" "X133980.bam" "X133981.bam" "X133982.bam" "X133983.bam"
[31] "X133984.bam" "X133985.bam" "X133986.bam" "X133987.bam" "X133989.bam"
[36] "X133990.bam" "X133992.bam" "X133993.bam" "X133997.bam" "X133998.bam"
[41] "X133999.bam" "X134000.bam" "X134001.bam" "X134003.bam" "X134004.bam"
[46] "X134005.bam" "X134007.bam" "X134008.bam" "X134011.bam" "X134012.bam"
[51] "X134013.bam" "X134015.bam" "X134018.bam" "X134020.bam" "X134720.bam"
[56] "X134721.bam" "X134722.bam" "X134723.bam" "X134724.bam" "X134725.bam"
[61] "X134726.bam" "X134729.bam" "X134730.bam" "X134731.bam" "X134732.bam"
[66] "X134735.bam" "X134736.bam" "X134737.bam" "X134738.bam" "X134739.bam"
[71] "X134742.bam" "X134743.bam" "X134744.bam" "X134745.bam" "X135606.bam"
[76] "X135607.bam" "X135608.bam" "X135609.bam" "X135610.bam" "X135611.bam"
[81] "X135612.bam"

[1] "A00_flg20_X133941" "A00_flg20_X133942" "A02_X133943"      
 [4] "A02_X133944"       "A02_X133945"       "A03_X133946"      
 [7] "A03_X133947"       "A04_X133949"       "A04_X133950"      
[10] "A04_X133951"       "A05_X133952"       "A05_X133954"      
[13] "DMSO_X133955"      "DMSO_X133956"      "DMSO_X133957"     
[16] "B01_X133961"       "B02_X133964"       "B02_X133965"      
[19] "B02_X133966"       "B04_X133967"       "B04_X133968"      
[22] "B04_X133969"       "C02_X133970"       "C02_X133971"      
[25] "C08_X133975"       "D07_X133976"       "D08_X133980"      
[28] "D08_X133981"       "E02_X133982"       "E02_X133983"      
[31] "E02_X133984"       "E06_X133985"       "E06_X133986"      
[34] "E06_X133987"       "E10_X133989"       "E10_X133990"      
[37] "E12_X133992"       "E12_X133993"       "F05_X133997"      
[40] "F05_X133998"       "F05_X133999"       "F09_X134000"      
[43] "F09_X134001"       "F12_X134003"       "F12_X134004"      
[46] "F12_X134005"       "G06_X134007"       "G06_X134008"      
[49] "G08_X134011"       "G10_X134012"       "G10_X134013"      
[52] "G11_X134015"       "Z00_flg22_X134018" "Z00_flg22_X134020"
[55] "A00_flg20_X134720" "A03_X134721"       "A05_X134722"      
[58] "A12_X134723"       "A12_X134724"       "A12_X134725"      
[61] "B01_X134726"       "C08_X134729"       "C08_X134730"      
[64] "D07_X134731"       "D07_X134732"       "E12_X134735"      
[67] "F03_X134736"       "F03_X134737"       "F03_X134738"      
[70] "F09_X134739"       "G08_X134742"       "G10_X134743"      
[73] "G11_X134744"       "G11_X134745"       "B01_X135606"      
[76] "C02_X135607"       "D08_X135608"       "E10_X135609"      
[79] "G06_X135610"       "G08_X135611"       "Z00_flg22_X135612"

[1] "DMSO_X133955"      "DMSO_X133956"      "DMSO_X133957"     
[4] "Z00_flg22_X134018" "Z00_flg22_X134020" "Z00_flg22_X135612"

,DMSO_X133955,DMSO_X133956,DMSO_X133957,Z00_flg22_X134018,Z00_flg22_X134020,Z00_flg22_X135612
AT1G01010,5276,3986,2540,2214,3526,3897
AT1G01020,2272,2077,1877,949,1052,1377
AT1G03987,31,19,28,9,3,14
AT1G01030,2000,1702,1339,516,267,300
AT1G01040,6000,4644,6160,2071,2395,3641
AT1G03993,0,0,0,0,0,0


In [6]:
### DEseq analysis -----
#define factor for DEseq
condition_flg22 <- factor(c(rep('DMSO', 3), rep('Z00_flg22', 3)))

#trim saminfo
saminfo_flg22_trim <- saminfo_flg22 %>%
  filter(grepl("^DMSO", condition) | grepl("^Z00_flg22", condition))

saminfo_flg22_trim
saminfo_flg22_trim$colnames_count <- colnames(countdata_flg22)
coldata_flg22 <- data.frame(row.names=colnames(countdata_flg22), condition_flg22)

#Chem screening
dds_flg22 <- DESeqDataSetFromMatrix(countData=countdata_flg22, colData=coldata_flg22, design=~condition_flg22)
dds_flg22 <- DESeq(dds_flg22)
colnames(dds_flg22) <- condition_flg22

res_flg22 <- results(dds_flg22, contrast=c('condition_flg22', 'Z00_flg22', 'DMSO'))

table(res_flg22$padj<0.1)

resDF_flg22<-as.data.frame(res_flg22)

output_flg22 <- merge(resDF_flg22, as.data.frame(counts(dds_flg22, normalized=TRUE)), by="row.names", sort=FALSE)

names(output_flg22)[1] <- "Gene"

output_flg22<-merge(GOdata, output_flg22, by="Gene")

write.csv(output_flg22, file="DESeq2/DEseq2_results_flg22_leaf(DESeq_DMSO_flg22_only_selected).csv")



sample,condition,index
<chr>,<chr>,<int>
X133955,DMSO_X133955,13
X133956,DMSO_X133956,14
X133957,DMSO_X133957,15
X134018,Z00_flg22_X134018,53
X134020,Z00_flg22_X134020,54
X135612,Z00_flg22_X135612,81


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




FALSE  TRUE 
12851  8896 

In [7]:
### Generate final table -----
Total <- output_flg22

colnames(Total)
Total <- Total %>%
  dplyr::select(Gene, geneabb, genename, log2FoldChange, padj,
                log2FoldChange, padj,
                starts_with("DMSO"), starts_with("Z00_flg22"),
                GO)
colnames(Total)

colnames(Total) <- c("Gene", "Gene_Symbol", "Gene_Name", 
                     "Log2FC_flg22", "Padj_flg22",
                     "DMSO_rep1", "DMSO_rep2", "DMSO_rep3",
                     "flg22_rep1", "flg22_rep2", "flg22_rep3",                     
                     "GO")
colnames(Total)

write.csv(Total, file="Total/Total_flg22_leaf(DESeq_DMSO_flg22_only_selected).csv")


[1] "Gene"           "geneabb"        "genename"       "GO"            
 [5] "baseMean"       "log2FoldChange" "lfcSE"          "stat"          
 [9] "pvalue"         "padj"           "DMSO"           "DMSO.1"        
[13] "DMSO.2"         "Z00_flg22"      "Z00_flg22.1"    "Z00_flg22.2"

[1] "Gene"           "geneabb"        "genename"       "log2FoldChange"
 [5] "padj"           "DMSO"           "DMSO.1"         "DMSO.2"        
 [9] "Z00_flg22"      "Z00_flg22.1"    "Z00_flg22.2"    "GO"

[1] "Gene"         "Gene_Symbol"  "Gene_Name"    "Log2FC_flg22" "Padj_flg22"  
 [6] "DMSO_rep1"    "DMSO_rep2"    "DMSO_rep3"    "flg22_rep1"   "flg22_rep2"  
[11] "flg22_rep3"   "GO"